In [ ]:
import cv2
import os
import pandas as pd

IMG_WIDTH=30
IMG_HEIGHT=30

# a method suitable for both training data and test data loading
def load_data(data_dir, csv_file_path):
    images,labels=[],[]
    df=pd.read_csv(csv_file_path)
    df=df[["ClassId","Path"]]
    for row in df.itertuples():
        image_path=os.path.join(data_dir, row.Path)
        image = cv2.imread(image_path)
        resized_image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        images.append(resized_image)
        labels.append(int(row.ClassId))
        '''print(images)
        print(type(images[0]))
        print(images[0].shape)
        print(labels)'''
    print("Data loaded")
    return (images,labels)

# loading the test and training data
def load_train_and_test_data(data_dir):
    train,test="Train.csv","Test.csv"
    files=set([f for f in os.listdir(data_dir)])
    if test in files:
        test_label_path=os.path.join(data_dir,test)
        x_train,y_train=load_data(data_dir,test_label_path)
    else:
        raise Exception("Test labels are not available")
    if train in files:
        train_label_path=os.path.join(data_dir,train)
        x_test,y_test=load_data(data_dir,train_label_path)
    else:
        raise Exception("Training labels are not available")
    return (x_train,y_train,x_test,y_test)


In [3]:
import numpy as np

def data_preprocessing(x_train,x_test):
    # mean subtraction and normalization
    mean = np.mean(x_train, axis=0)
    std = np.std(x_train, axis=0)
    x_train = (x_train - mean) / std
    x_test = (x_test - mean) / std
    print("Data preprocessing completed.")
    return (x_train,x_test)

In [4]:
import tensorflow as tf
import numpy as np
import sys
import pandas as pd
from model_training import get_model
from data_preprocessing import data_preprocessing
from data_loading import load_train_and_test_data

EPOCHS = 10
num_of_categories=43
IMG_WIDTH=30
IMG_HEIGHT=30

# load the train and test data
x_train, y_train, x_test, y_test = load_train_and_test_data("data")
x_train, y_train, x_test, y_test = np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)
# one hot encoding
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
x_train, x_test=data_preprocessing(x_train, x_test)

Data loaded
Data loaded
Data preprocessing completed.


In [12]:
import tensorflow as tf
num_of_categories=43
IMG_WIDTH=30
IMG_HEIGHT=30
def get_model(regularizer_strength,dropout_rate):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(regularizer_strength), input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(regularizer_strength), kernel_initializer='he_normal'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation="relu", kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(regularizer_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.Dense(num_of_categories, activation="softmax")
    ])
    model.compile(
        optimizer="nadam",  
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    print("Model trained.")
    return model


In [13]:
#test the best setup for dropout_rates and regularizer_strengths
dropout_rates=[i/20 for i in range (1,16)]
regularizer_strengths=[i/20 for i in range (1,13)]
accuracies=[]

for dropout_rate in dropout_rates:
    for regularizer_strength in regularizer_strengths:        
        model = get_model(dropout_rate,regularizer_strength)

        model.fit(x_train, y_train, epochs=EPOCHS, batch_size=32)

        # Evaluate neural network performance
        loss,accuracy=model.evaluate(x_test, y_test, verbose=2, batch_size=32)
        accuracies.append([dropout_rate,regularizer_strength,accuracies])

c:\Users\smile\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model trained.
Epoch 1/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 49s 105ms/step - accuracy: 0.7008 - loss: 11.3061
Epoch 2/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 87s 118ms/step - accuracy: 0.8855 - loss: 2.5037
Epoch 3/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 45s 115ms/step - accuracy: 0.8995 - loss: 2.1773
Epoch 4/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 38s 96ms/step - accuracy: 0.9120 - loss: 1.9594
Epoch 5/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 44s 112ms/step - accuracy: 0.9154 - loss: 1.8527
Epoch 6/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 42s 107ms/step - accuracy: 0.9199 - loss: 1.6369
Epoch 7/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 48s 121ms/step - accuracy: 0.9295 - loss: 1.3872
Epoch 8/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 47s 118ms/step - accuracy: 0.9302 - loss: 1.2850
Epoch 9/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 51s 129ms/step - accuracy: 0.9434 - loss: 1.0841
Epoch 10/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 46s 117ms/step - accuracy: 0.9460 - loss: 0.9824
1226/1226 - 30s - 24ms/step - accuracy: 0.8226 - loss: 1.5691
Model trained.
Epoch 1/10
395/39

MemoryError: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Your list of [x, y, z = f(x,y)]
accuracies = np.array(accuracies)

# Extract unique x and y values
x_vals = np.unique(accuracies[:, 0])
y_vals = np.unique(accuracies[:, 1])

# Reshape z values into a 2D grid
z_vals = accuracies[:, 2].reshape(len(y_vals), len(x_vals))

# Plot heatmap
plt.imshow(z_vals, 
           extent=[x_vals.min(), x_vals.max(), y_vals.min(), y_vals.max()],
           origin='lower', 
           aspect='auto',
           cmap='viridis')  # or 'hot', 'plasma', etc.

plt.colorbar(label='accu(drop,reg)')
plt.xlabel('dropout_rate')
plt.ylabel('regularizer_strength')
plt.title('accuracies')
plt.show()